In [1]:
%matplotlib inline

In [2]:
from datetime import datetime

In [3]:
#renderer
from PIL import ImageFont
import numpy as np
import cv2
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_

char_size = 24
# char render
def render(text, font=None):
    if font is None:
        font = ImageFont.truetype("/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc", char_size)
    mask = font.getmask(text)
    size = mask.size[::-1]
    a = np.asarray(mask).reshape(size) / 255
    res = cv2.resize(a, dsize=(char_size, char_size), interpolation=cv2.INTER_CUBIC)
    return res

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/data_utils.py
import torch
import re

class Dictionary(object):
    def __init__(self, max_size=None):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 1
        self.word2idx['⸘'] = 0 # as unk
        self.idx2word[0] = '⸘'
        self.max_size = max_size + 1
    
    def add_word(self, word):
        if not word in self.word2idx and self.idx < self.max_size:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    def __init__(self, max_size=None):
        self.dictionary = Dictionary(max_size=max_size)

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
#         with open(path, 'r') as f:
#             tokens = 0
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 tokens += len(words)
#                 for word in words: 
#                     self.dictionary.add_word(word)  

        # split words to char and add to dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                tokens += len(chars)
                for char in chars:
                    self.dictionary.add_word(char)
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                for char in chars:
                    if char in self.dictionary.word2idx:
                        ids[token] = self.dictionary.word2idx[char]
                        token += 1
                    else:
                        ids[token] = self.dictionary.word2idx['⸘']
                        token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, char_id, char_cnn_o, h):
        # Embed word ids to vectors
        x = self.embed(char_id) + char_cnn_o
        
        # Forward propagate LSTM
        out, h = self.gru(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, h

In [8]:
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    if type(stride) is not tuple:
        stride = (stride, stride)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride[0]) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride[1]) + 1)
    return h, w

# Dai et al. 's CNN glyph encoder
class Dai_CNN(nn.Module):
    def __init__(self, embed_size, input_size=(24, 24)):
        super(Dai_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, (7, 7), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)
        h, w = conv_output_shape(input_size, (7, 7), (2, 2))
        
        self.conv2 = nn.Conv2d(32, 16, (5, 5), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)
        h, w = conv_output_shape((h, w), (5, 5), (2, 2))
                
        self.fc = nn.Linear(16*h*w, embed_size)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.zeros_(self.fc.bias)
        
        self.h, self.w = h, w
        
    def forward(self, char_img):
        b = char_img.size(0)
        x = self.conv1(char_img)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = x.view(-1, 16*self.h*self.w)
        return self.fc(x)

In [9]:
# Hyper-parameters
embed_size = 300
hidden_size = 128
num_layers = 1
num_epochs = 50
batch_size = 16
seq_length = 32
learning_rate = 1e-3

# Load dataset
corpus = Corpus(max_size=4000)
ids = corpus.get_data('icwb2-data/training/msr_training.utf8', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [10]:
# save char images for reference
for char, idx in corpus.dictionary.word2idx.items():
    np.save(f'char_img/noto_CJK/msr3/{idx}.npy', render(char))

In [11]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
cnn_encoder = Dai_CNN(embed_size, input_size=(char_size, char_size)).to(device)
model.train()
cnn_encoder.train()
params = list(model.parameters())+list(cnn_encoder.parameters())

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Truncated backpropagation
def detach(state):
    return state.detach()

In [12]:
# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Get images
        images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
        for b, seq in enumerate(ids):
            for s, idx in enumerate(ids[b][i:i+seq_length]):
                images[b, s] = np.load(f'char_img/noto_CJK/msr3/{idx}.npy').reshape(char_size,char_size,1)
        images = torch.from_numpy(images).float().to(device) # B N H W C
        images = images.view(-1, char_size, char_size, 1) # B*N H W C
        images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
        
        # Get encoded images
        cnn_o = cnn_encoder(images)
        cnn_o = cnn_o.view(inputs.size(0), inputs.size(1), -1)
               
        # Forward pass
        state = detach(state)
        outputs, state = model(inputs, cnn_o, state)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        cnn_encoder.zero_grad()
        loss.backward()
        clip_grad_norm_(params, 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 1000 == 0:
            print ('{} Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(datetime.now(), epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

2019-11-25 16:34:54.712081 Epoch [1/50], Step[0/8080], Loss: 8.3154, Perplexity: 4086.51
2019-11-25 16:39:45.120695 Epoch [1/50], Step[1000/8080], Loss: 5.1071, Perplexity: 165.19
2019-11-25 16:44:40.094791 Epoch [1/50], Step[2000/8080], Loss: 4.5968, Perplexity: 99.17
2019-11-25 16:50:47.388058 Epoch [1/50], Step[3000/8080], Loss: 4.4296, Perplexity: 83.89
2019-11-25 16:56:59.091131 Epoch [1/50], Step[4000/8080], Loss: 4.2543, Perplexity: 70.41
2019-11-25 17:02:48.925007 Epoch [1/50], Step[5000/8080], Loss: 4.7234, Perplexity: 112.55
2019-11-25 17:08:18.938422 Epoch [1/50], Step[6000/8080], Loss: 4.5673, Perplexity: 96.28
2019-11-25 17:14:35.499657 Epoch [1/50], Step[7000/8080], Loss: 4.1168, Perplexity: 61.36
2019-11-25 17:20:52.902418 Epoch [1/50], Step[8000/8080], Loss: 4.3748, Perplexity: 79.43
2019-11-25 17:21:23.394436 Epoch [2/50], Step[0/8080], Loss: 4.2921, Perplexity: 73.12
2019-11-25 17:27:44.810220 Epoch [2/50], Step[1000/8080], Loss: 4.1367, Perplexity: 62.59
2019-11-25 1

2019-11-26 01:30:16.915658 Epoch [11/50], Step[2000/8080], Loss: 3.9425, Perplexity: 51.55
2019-11-26 01:36:52.826100 Epoch [11/50], Step[3000/8080], Loss: 3.8018, Perplexity: 44.78
2019-11-26 01:43:34.879967 Epoch [11/50], Step[4000/8080], Loss: 3.6738, Perplexity: 39.40
2019-11-26 01:50:13.777066 Epoch [11/50], Step[5000/8080], Loss: 4.3004, Perplexity: 73.73
2019-11-26 01:56:53.495094 Epoch [11/50], Step[6000/8080], Loss: 3.9745, Perplexity: 53.22
2019-11-26 02:03:35.639254 Epoch [11/50], Step[7000/8080], Loss: 3.6711, Perplexity: 39.30
2019-11-26 02:10:17.674630 Epoch [11/50], Step[8000/8080], Loss: 3.9831, Perplexity: 53.68
2019-11-26 02:10:50.021813 Epoch [12/50], Step[0/8080], Loss: 3.8599, Perplexity: 47.46
2019-11-26 02:17:31.057327 Epoch [12/50], Step[1000/8080], Loss: 3.7524, Perplexity: 42.62
2019-11-26 02:24:13.188626 Epoch [12/50], Step[2000/8080], Loss: 3.9298, Perplexity: 50.90
2019-11-26 02:30:53.917969 Epoch [12/50], Step[3000/8080], Loss: 3.7845, Perplexity: 44.01
20

2019-11-26 10:35:55.389299 Epoch [21/50], Step[3000/8080], Loss: 3.7121, Perplexity: 40.94
2019-11-26 10:42:33.039552 Epoch [21/50], Step[4000/8080], Loss: 3.6738, Perplexity: 39.40
2019-11-26 10:49:13.183945 Epoch [21/50], Step[5000/8080], Loss: 4.2801, Perplexity: 72.24
2019-11-26 10:55:54.588817 Epoch [21/50], Step[6000/8080], Loss: 3.8493, Perplexity: 46.96
2019-11-26 11:02:35.853341 Epoch [21/50], Step[7000/8080], Loss: 3.6576, Perplexity: 38.77
2019-11-26 11:09:16.094941 Epoch [21/50], Step[8000/8080], Loss: 3.9380, Perplexity: 51.31
2019-11-26 11:09:48.166277 Epoch [22/50], Step[0/8080], Loss: 3.7851, Perplexity: 44.04
2019-11-26 11:16:28.177199 Epoch [22/50], Step[1000/8080], Loss: 3.6933, Perplexity: 40.18
2019-11-26 11:23:05.061838 Epoch [22/50], Step[2000/8080], Loss: 3.8816, Perplexity: 48.50
2019-11-26 11:29:47.394436 Epoch [22/50], Step[3000/8080], Loss: 3.7296, Perplexity: 41.66
2019-11-26 11:36:30.857320 Epoch [22/50], Step[4000/8080], Loss: 3.6752, Perplexity: 39.46
20

2019-11-26 19:47:01.003340 Epoch [31/50], Step[4000/8080], Loss: 3.6263, Perplexity: 37.58
2019-11-26 19:53:45.440191 Epoch [31/50], Step[5000/8080], Loss: 4.2611, Perplexity: 70.89
2019-11-26 20:00:33.066453 Epoch [31/50], Step[6000/8080], Loss: 3.8422, Perplexity: 46.63
2019-11-26 20:07:20.864144 Epoch [31/50], Step[7000/8080], Loss: 3.6490, Perplexity: 38.44
2019-11-26 20:14:08.269034 Epoch [31/50], Step[8000/8080], Loss: 3.9097, Perplexity: 49.88
2019-11-26 20:14:40.858352 Epoch [32/50], Step[0/8080], Loss: 3.7812, Perplexity: 43.87
2019-11-26 20:21:25.474911 Epoch [32/50], Step[1000/8080], Loss: 3.6765, Perplexity: 39.51
2019-11-26 20:28:10.779424 Epoch [32/50], Step[2000/8080], Loss: 3.9093, Perplexity: 49.86
2019-11-26 20:34:56.354798 Epoch [32/50], Step[3000/8080], Loss: 3.7224, Perplexity: 41.36
2019-11-26 20:41:48.670083 Epoch [32/50], Step[4000/8080], Loss: 3.6395, Perplexity: 38.07
2019-11-26 20:48:36.054948 Epoch [32/50], Step[5000/8080], Loss: 4.2555, Perplexity: 70.49
20

2019-11-27 05:03:13.927036 Epoch [41/50], Step[5000/8080], Loss: 4.2305, Perplexity: 68.75
2019-11-27 05:09:58.084271 Epoch [41/50], Step[6000/8080], Loss: 3.8512, Perplexity: 47.05
2019-11-27 05:16:33.892985 Epoch [41/50], Step[7000/8080], Loss: 3.5891, Perplexity: 36.20
2019-11-27 05:23:08.676334 Epoch [41/50], Step[8000/8080], Loss: 3.9186, Perplexity: 50.33
2019-11-27 05:23:40.199309 Epoch [42/50], Step[0/8080], Loss: 3.7590, Perplexity: 42.91
2019-11-27 05:30:20.809142 Epoch [42/50], Step[1000/8080], Loss: 3.6503, Perplexity: 38.49
2019-11-27 05:37:00.245113 Epoch [42/50], Step[2000/8080], Loss: 3.8934, Perplexity: 49.08
2019-11-27 05:43:36.355901 Epoch [42/50], Step[3000/8080], Loss: 3.6952, Perplexity: 40.26
2019-11-27 05:50:15.368253 Epoch [42/50], Step[4000/8080], Loss: 3.6649, Perplexity: 39.05
2019-11-27 05:56:51.995562 Epoch [42/50], Step[5000/8080], Loss: 4.2587, Perplexity: 70.71
2019-11-27 06:03:26.656049 Epoch [42/50], Step[6000/8080], Loss: 3.8684, Perplexity: 47.87
20

In [13]:
model.eval()
cnn_encoder.eval()

perplexity = .0
num_step = 0
for i in range(0, ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = ids[:, i:i+seq_length].to(device)
    targets = ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
    for b, seq in enumerate(ids):
        for s, idx in enumerate(ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr3/{idx}.npy').reshape(char_size,char_size,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, char_size, char_size, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Get encoded images
    cnn_o = cnn_encoder(images)
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(inputs, cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))
    
    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Train Perplexity: {perplexity / num_step}")

Train Perplexity: 43.938697216289434


In [14]:
test_ids = corpus.get_data('icwb2-data/testing/msr_test.utf8', batch_size)
# filter out unknown character
test_ids = test_ids.view(-1)
mask = test_ids < vocab_size
test_ids = test_ids[mask]
num_batches = test_ids.size(0) // batch_size
test_ids = test_ids[:num_batches*batch_size]
test_ids = test_ids.view(batch_size, -1)

In [15]:
perplexity = .0
num_step = 0
for i in range(0, test_ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = test_ids[:, i:i+seq_length].to(device)
    targets = test_ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
    for b, seq in enumerate(test_ids):
        for s, idx in enumerate(test_ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr3/{idx}.npy').reshape(char_size,char_size,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, char_size, char_size, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Get encoded images
    cnn_o = cnn_encoder(images)
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(inputs, cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))

    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Test Perplexity: {perplexity / num_step}")

Test Perplexity: 58.1385755453622
